<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6008b0c331ea8ca64434712b348543b6c407ed7fd6d8ab6752ea638eacb08777
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 10:04:12
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: -1.25 L (-0.89%)
Current PnL: -22.44 L (-14.95%)
CY Booked + Current PnL: -10.54 L (-7.02%)
-------------------
Total profit:  1.37 L
Total loss:  -23.80 L
-------------------
Total Booked + Current PnL: 16.44 L (13.36%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.56%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.64 L (64.47%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.25%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.47,55.0,X-LC,3.74,212460.0,7960.0,15340.0,-0.82,3.89,7.22,11.39,10.0,0.52,1.54,13.19,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,94.02,42.0,M-SC,1.37,83552.0,-17225.0,17320.0,0.06,-17.09,20.73,0.09,245.0,-0.99,0.61,9.77,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.66,53.0,X-MC,1.03,208410.0,16668.0,21091.0,0.79,8.69,10.12,19.69,99.0,0.79,1.51,16.08,XY25,NTT,AC
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.85,214734.0,4368.0,24415.0,0.11,2.08,11.37,13.68,37.0,0.18,1.56,18.94,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194769.0,-5369.0,26255.0,-0.24,-2.68,13.48,10.44,4.0,-0.20,1.41,2.73,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.00,12.24,66.0,X-SC,10.28,198768.0,7762.0,49931.0,3.56,4.06,25.12,30.21,157.0,0.16,1.44,25.79,X40N,NTT,FINANCE
31,HINDZINC,730.22,38.75,71.0,M-LC,8.73,226287.0,21211.0,91352.0,2.90,10.34,40.37,54.89,52.0,0.23,1.64,36.93,X5K,ATH,METALS
34,ICICIPRULI,790.00,-20.88,51.0,X-MC,1.93,136165.0,377.0,42375.0,1.50,0.28,31.12,31.48,107.0,0.01,0.99,12.43,X40,ATH,INSURANCE
47,LTIM,7230.20,1.16,69.0,H-LC,3.12,250988.0,4837.0,74368.0,1.45,1.96,29.63,32.18,16.0,0.07,1.82,38.72,X200,ATH,IT
17,CERA,9475.00,-23.15,43.0,H-SC,1.97,139955.0,-35948.0,77969.0,1.21,-20.44,55.71,23.89,149.0,-0.46,1.01,20.87,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-55.15,11.0,X-LC,1.12,142278.0,-134245.0,244320.0,-40.68,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
53,RAJOOENG,143.1,-42.97,41.0,H-SC,17.38,86550.0,-15950.0,56552.0,-3.26,-15.56,65.34,39.61,114.0,-0.28,0.63,5.70,AR,ATH,MISC
81,VALIANTORG,838.0,-281.53,24.0,H-SC,3.04,100997.0,-62608.0,183926.0,-2.77,-38.27,182.11,74.15,139.0,-0.34,0.73,29.46,XR,NTT,CHEMICALS
10,BANDHANBNK,400.0,-11.95,51.0,H-SC,2.99,220899.0,-57660.0,315090.0,-2.15,-20.70,142.64,92.42,151.0,-0.18,1.60,27.66,XY24,NTT,BANKS
68,SURYODAY,240.0,60.09,56.0,H-SC,10.53,147390.0,-31681.0,97410.0,-1.95,-17.69,66.09,36.71,135.0,-0.33,1.07,46.24,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.0,45.36,43.0,M-LC,6.2,203472.0,302.0,41834.0,-0.31,0.15,20.56,20.74,55.0,0.01,1.47,5.70,XY25,NTT,FINANCE
85,WIPRO,420.0,-13.34,49.0,M-LC,6.0,152098.0,1153.0,108294.0,0.08,0.76,71.20,72.51,53.0,0.01,1.10,6.83,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,62.0,H-SC,11.64,131220.0,-11259.0,30784.0,-0.29,-7.90,23.46,13.70,163.0,-0.37,0.95,54.88,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,94.02,42.0,M-SC,1.37,83552.0,-17225.0,17320.0,0.06,-17.09,20.73,0.09,245.0,-0.99,0.61,9.77,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220707.0,-48960.0,85303.0,-0.19,-18.16,38.65,13.47,138.0,-0.57,1.60,12.22,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,1.97,139955.0,-35948.0,77969.0,1.21,-20.44,55.71,23.89,149.0,-0.46,1.01,20.87,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,43.0,H-MC,7.01,101698.0,-30137.0,72358.0,0.01,-22.86,71.15,32.03,98.0,-0.42,0.74,13.91,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203472.0,302.0,41834.0,-0.31,0.15,20.56,20.74,55.0,0.01,1.47,5.70,XY25,NTT,FINANCE
59,SAIL,228.00,44.35,50.0,M-MC,12.61,227425.0,2463.0,165202.0,-0.33,1.09,72.64,74.53,192.0,0.01,1.65,33.70,XY24,BTT,STEEL
31,HINDZINC,730.22,38.75,71.0,M-LC,8.73,226287.0,21211.0,91352.0,2.90,10.34,40.37,54.89,52.0,0.23,1.64,36.93,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,55.0,M-LC,9.56,157702.0,16862.0,27629.0,-0.86,11.97,17.52,31.59,66.0,0.61,1.14,41.63,XY24,NTT,BANKS
85,WIPRO,420.00,-13.34,49.0,M-LC,6.00,152098.0,1153.0,108294.0,0.08,0.76,71.20,72.51,53.0,0.01,1.10,6.83,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,38.75,71.0,M-LC,8.73,226287.0,21211.0,91352.0,2.90,10.34,40.37,54.89,52.0,0.23,1.64,36.93,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,62.0,H-MC,12.71,193004.0,13196.0,99223.0,-1.08,7.34,51.41,62.52,88.0,0.13,1.40,43.93,XR,NTT,BANKS
36,INDIAMART,4810.62,-55.20,39.0,H-SC,8.32,125329.0,1993.0,129628.0,-1.41,1.62,103.43,106.72,119.0,0.02,0.91,25.37,AR,ATH,MISC
85,WIPRO,420.00,-13.34,49.0,M-LC,6.00,152098.0,1153.0,108294.0,0.08,0.76,71.20,72.51,53.0,0.01,1.10,6.83,XR,NTT,IT
84,WHIRLPOOL,2270.00,-47.86,35.0,M-SC,3.36,88132.0,-3366.0,82121.0,0.94,-3.68,93.18,86.07,223.0,-0.04,0.64,27.98,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142278.0,-134245.0,244320.0,-40.68,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,31.0,X-SC,37.82,51903.0,-13103.0,164029.0,-0.63,-20.16,316.03,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-7.74,32.0,X-MC,1.32,228712.0,-19019.0,94069.0,-0.48,-7.68,41.13,30.30,92.0,-0.20,1.66,4.59,X40,ATH,ELECTRICAL
5,ASIANPAINT,3465.66,-18.42,33.0,X-LC,5.33,205814.0,-45954.0,99161.0,-0.16,-18.25,48.18,21.13,27.0,-0.46,1.49,10.71,X40,ATH,PAINTS
22,DABUR,735.00,-12.84,34.0,X-MC,3.39,235904.0,-12164.0,120571.0,-0.28,-4.90,51.11,43.70,102.0,-0.10,1.71,8.83,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.47,42.0,H-LC,0.93,154995.0,-31100.0,78582.0,0.73,-16.71,50.70,25.51,15.0,-0.40,1.12,14.22,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.66,53.0,X-MC,1.03,208410.0,16668.0,21091.0,0.79,8.69,10.12,19.69,99.0,0.79,1.51,16.08,XY25,NTT,AC
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142278.0,-134245.0,244320.0,-40.68,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-7.74,32.0,X-MC,1.32,228712.0,-19019.0,94069.0,-0.48,-7.68,41.13,30.30,92.0,-0.20,1.66,4.59,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.81,40.0,X-MC,1.46,136212.0,79.0,31043.0,0.40,0.06,22.79,22.86,91.0,0.00,0.99,12.68,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,31.0,X-SC,37.82,51903.0,-13103.0,164029.0,-0.63,-20.16,316.03,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-55.15,11.0,X-LC,1.12,142278.0,-134245.0,244320.0,-40.68,-48.55,171.72,39.81,54.0,-0.55,1.03,0.00,XY24,NTT,AUTO
82,VBL,671.64,-21.85,37.0,X-LC,10.51,281422.0,-34420.0,147071.0,-0.31,-10.90,52.26,35.67,5.0,-0.23,2.04,1.45,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.42,36.0,X-MC,7.67,220700.0,-42665.0,156013.0,-0.64,-16.20,70.69,43.04,84.0,-0.27,1.60,2.46,XY25,ATH,FMCG
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194769.0,-5369.0,26255.0,-0.24,-2.68,13.48,10.44,4.0,-0.20,1.41,2.73,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,31.0,X-SC,37.82,51903.0,-13103.0,164029.0,-0.63,-20.16,316.03,232.18,198.0,-0.08,0.38,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-40.36,37.0,X-SC,3.27,81282.0,-63878.0,139805.0,0.09,-44.01,172.00,52.31,136.0,-0.46,0.59,8.80,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.58,37.0,X-SC,4.33,94844.0,-33826.0,79128.0,-0.55,-26.29,83.43,35.20,219.0,-0.43,0.69,8.86,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-3.61,56.0,X-SC,1.01,109536.0,7532.0,36388.0,1.13,7.38,33.22,43.06,122.0,0.21,0.79,25.76,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.22,108450.0,-19488.0,66621.0,-0.01,-15.23,61.43,36.84,143.0,-0.29,0.79,11.03,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-29.33,47.0,X-LC,12.27,282066.0,-63890.0,132120.0,-0.22,-18.47,46.84,19.73,1.0,-0.48,2.04,3.89,X40,ATH,IT
39,INFY,2275.00,-18.52,52.0,X-LC,7.63,318052.0,5164.0,166532.0,-1.43,1.65,52.36,54.87,3.0,0.03,2.31,8.41,X40,BTT,IT
41,ITC,452.00,-39.61,40.0,X-LC,2.74,194769.0,-5369.0,26255.0,-0.24,-2.68,13.48,10.44,4.0,-0.20,1.41,2.73,X40,NTT,FMCG
82,VBL,671.64,-21.85,37.0,X-LC,10.51,281422.0,-34420.0,147071.0,-0.31,-10.90,52.26,35.67,5.0,-0.23,2.04,1.45,X40N,ATH,FMCG
1,ABB,7934.00,-41.13,41.0,H-LC,7.28,246024.0,-15595.0,134797.0,-0.43,-5.96,54.79,45.57,7.0,-0.12,1.78,4.69,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.72,81120.0,-32429.0,72481.0,-0.80,-28.56,89.35,35.27,268.0,-0.45,0.59,98.54,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.92,89865.0,-8115.0,29718.0,-0.76,-8.28,33.07,22.05,152.0,-0.27,0.65,30.63,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,62.0,H-SC,11.64,131220.0,-11259.0,30784.0,-0.29,-7.90,23.46,13.70,163.0,-0.37,0.95,54.88,OX40N,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.46,147177.0,-2043.0,146220.0,0.69,-1.37,99.35,96.62,208.0,-0.01,1.07,59.28,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.53,147390.0,-31681.0,97410.0,-1.95,-17.69,66.09,36.71,135.0,-0.33,1.07,46.24,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,66.0,M-SC,17.46,147177.0,-2043.0,146220.0,0.69,-1.37,99.35,96.62,208.0,-0.01,1.07,59.28,XY25,NTT,FINANCE
68,SURYODAY,240.00,60.09,56.0,H-SC,10.53,147390.0,-31681.0,97410.0,-1.95,-17.69,66.09,36.71,135.0,-0.33,1.07,46.24,XR,NTT,BANKS
31,HINDZINC,730.22,38.75,71.0,M-LC,8.73,226287.0,21211.0,91352.0,2.90,10.34,40.37,54.89,52.0,0.23,1.64,36.93,X5K,ATH,METALS
63,SHALBY,327.00,1179.11,55.0,M-SC,22.97,173874.0,-7269.0,52736.0,-1.55,-4.01,30.33,25.10,235.0,-0.14,1.26,39.34,XY24,NTT,HEALTHCARE
4,ANGELONE,3033.00,12.24,66.0,X-SC,10.28,198768.0,7762.0,49931.0,3.56,4.06,25.12,30.21,157.0,0.16,1.44,25.79,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.45
1,25,44.00
2,50,75.76


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    33.63
MC    21.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.67
X40      14.84
XY25     12.07
XR       10.99
X40N     10.12
AR        8.19
OX40N     7.70
X200      1.82
X5K       1.64
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.67
X-LC    22.53
X-MC    16.05
M-SC    12.74
M-LC     5.35
H-LC     4.72
X-SC     4.68
H-MC     3.81
M-MC     1.65
L-SC     1.45
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.97,-6.50,41.60
IT,13.05,-16.73,79.96
FINANCE,12.18,-12.41,63.37
MISC,6.97,-19.65,85.94
BANKS,6.47,-11.73,70.51
PAINTS,5.59,-19.66,37.50
ELECTRICAL,5.35,-11.54,50.67
INSURANCE,3.85,-4.36,43.37
AUTO,2.79,-49.43,120.86


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3341015.0,22
XR,1393765.0,14
AR,1198976.0,9
X40,793712.0,10
X40N,630816.0,8
OX40N,584951.0,10
XY25,518828.0,8
SR,266506.0,2
X5K,91352.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3457055.0,25
M-SC,1286244.0,15
X-LC,1133593.0,13
X-MC,1111446.0,11
X-SC,535902.0,6
H-MC,354440.0,3
H-LC,287747.0,3
M-LC,269109.0,4
L-SC,261461.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1204410.0      6
           XR         823620.0      7
           AR         820826.0      5
M-SC       XY24       693052.0      6
X-MC       XY24       591866.0      4
X-LC       X40        491178.0      5
           XY24       311968.0      2
X-SC       X40N       292745.0      4
H-SC       OX40N      271931.0      4
           SR         266506.0      2
X-MC       X40        223406.0      4
X-LC       X40N       219001.0      3
H-LC       AR         213379.0      2
H-MC       XY24       182859.0      1
X-MC       XY25       177104.0      2
L-SC       XR         171701.0      2
M-MC       XY24       165202.0      1
M-SC       AR         164771.0      2
X-SC       XY24       164029.0      1
M-SC       OX40N      150902.0      4
           XY25       146220.0      1
           XR         131299.0      2
X-MC       X40N       119070.0      1
X-LC       XY25       111446.0      3
M-LC       XR         108294.0      1
H-MC       XR          99223.0      1
M-LC       X5K         91352.0      1
L-SC       OX40N       89760.0      1
X-SC       X40         79128.0      1
H-LC       X200        74368.0      1
H-MC       OX40N       72358.0      1
H-SC       MH          69762.0      1
L-MC       XR          59628.0      1
L-LC       XY25        42224.0      1
M-LC       XY25        41834.0      1
           XY24        27629.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
